In [ ]:
import gftool as gt
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# BEB algorithm for nonlocal disorder

The *BEB* [1,2] algorithm is an extension of *CPA* to include nonocal disorder;
that is, the hopping parameters depend on the components.

We combined the *BEB* algorithm with *DMFT* in the same fashion as *CPA+DMFT*.
While the *DMFT* part is very heavy, the disorder part can be easily computed.

This notebook can be used to play around with *BEB* using the algorithm given in [3].
It is given in the Python package `gftool`, you can find the documentation on [ReadTheDocs](https://gftools.readthedocs.io/en/stable/generated/gftool.beb.html).

[1] J. A. Blackman, D. M. Esterling, and N. F. Berk,
Phys. Rev. B **4**, 2412 (1971).  
DOI: [10.1103/PhysRevB.4.2412](https://doi.org/10.1103/PhysRevB.4.2412)

[2] D. M. Esterling,
Phys. Rev. B **12**, 1596 (1975).  
DOI: [10.1103/PhysRevB.12.1596](https://doi.org/10.1103/PhysRevB.12.1596)

[3] A. Weh, Y. Zhang, A. Östlin, H. Terletska, D. Bauernfeind, K.-M. Tam, H. G. Evertz, K. Byczuk, D. Vollhardt, and L. Chioncel,
Phys. Rev. B **104**, 045127  
DOI: [10.1103/PhysRevB.104.045127](https://doi.org/10.1103/PhysRevB.104.045127)


## Required parameters

The model is defined using the following parameters:

* `e_onsite`: The on-site energies $\underline{v}^\alpha$ of the components $\alpha$.

* `concentration`: The concentrations $c^\alpha$ of the components $\alpha$.

* `hopping`: The dimensionless hopping parameters $\underline{T}^{\alpha\beta}$ describing the hopping between components $\alpha$ and $\beta$.

* `hilbert_trafo`: The lattice Hilbert transformation of the lattice $$g_0(z) = \frac{1}{N} \sum_k \frac{1}{z - \epsilon_k} = \int d\epsilon \frac{1}{z - \epsilon}$$

Furthermore, we have to specify the frequencies, `z`, at which we evaluate the function.
To avoid singularities and branch cuts, the imaginary part `z.imag` should be finite.

Let's do an example calculation:

In [ ]:
from functools import partial  # partial fixes input arguements of a function

# Parameters of the setup
e_onsite = np.array([0.0, 0.3])
concentration = np.array([0.9, 0.1])
hopping = np.array([[1.0, 1.5],
                    [1.5, 1.0]])
hilbert_trafo = partial(gt.lattice.bethe.hilbert_transform, half_bandwidth=1.0)

# let's evaluate it on a contour parallel to the real axis.
ww = np.linspace(-4.0, 4.0, num=301) + 1e-6j

First, we self-consistently calculate the effective medium $\underline{S}^{\alpha\beta}$:

In [ ]:
self_beb = gt.beb.solve_root(
    ww, e_onsite, concentration,
    hopping=hopping, hilbert_trafo=hilbert_trafo,
)

With the effective medium, we can calculate the effective local Green's function $\underline{g}_\mathrm{loc}^{\alpha\alpha}$

In [ ]:
gf_loc = gt.beb.gf_loc_z(ww, self_beb, hopping=hopping, hilbert_trafo=hilbert_trafo)

plt.plot(ww.real, -1/np.pi*gf_loc[:, 0].imag, label=r"$\alpha=$A")
plt.plot(ww.real, -1/np.pi*gf_loc[:, 1].imag, label=r"$\alpha=$B")
plt.xlim(ww.real.min(), ww.real.max())
plt.ylabel(r"A̲$^\alpha$")
plt.tight_layout()
plt.legend()
plt.show()

The components of the effective local Green's $\underline{g}_\mathrm{loc}^{\alpha\alpha}$ function are the conditional expectation value times the corresponding concentration.
Thus, these Green's functions components are normalized to the concentration $c^\alpha$ and not to $1$.

The average Green's function is just the sum or components, $\mathbb{E}(G) = \sum_\alpha \underline{g}_\mathrm{loc}^{\alpha\alpha}$:

In [ ]:
plt.plot(ww.real, -1/np.pi*gf_loc.sum(axis=-1).imag)
plt.xlim(ww.real.min(), ww.real.max())
plt.ylabel(r"$\mathbb{E}(A)$")
plt.tight_layout()
plt.show()

Of course, we are not restricted to only two components or a Bethe lattice, available lattices can be listed with the following command:

In [ ]:
[lat for lat in dir(gt.lattice) if not lat.startswith('__')]

Depending on the problem, the self-consistency `gt.beb.solve_root` might be slow.
To show additional output, enable the logger on the desired level (this might however be verbose).

In [ ]:
# import logging

# logging.basicConfig()
# # logging.getLogger('gftool').setLevel(logging.INFO)
# # logging.getLogger('gftool').setLevel(logging.DEBUG)